https://python.langchain.com/v0.1/docs/integrations/tools/

In [1]:
!pip install arxiv

### wikipedia Tool

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wraper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

In [17]:
wiki = WikipediaQueryRun(api_wrapper= api_wraper)

In [18]:
wiki.name

'wikipedia'

### retriversearch 'langsmith_search tool'

### custom tool

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
loader = WebBaseLoader("http://docs.smith.langchain.com/")
docs= loader.load()

recursive_documnet = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap= 200).split_documents(docs)



In [10]:
vectordb= FAISS.from_documents(recursive_documnet, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

/Users/rasikagulhane/Desktop/LangChain/LCenv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1293f8700>)

In [25]:
from langchain.tools.retriever import create_retriever_tool
custom_tools= create_retriever_tool(retriever,"langsmith_search","search for any imformation about.For search of any question about langsmith, you must use this tools ")

In [26]:
custom_tools.name

'langsmith_search'

### Arxiv tool

In [23]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


api_wraper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper= api_wraper)

In [20]:
arxiv.name

'arxiv'

### Combine tools

In [27]:
tools =[wiki, custom_tools, arxiv]

## Query from this tools with agents

it will search query from each tools and give result

In [29]:
import os

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

In [30]:
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature = 0)

In [31]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
# Agent


from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)


In [33]:
from langchain.agents import AgentExecutor


agent_executor= AgentExecutor(agent = agent, tools = tools, verbose=True)


In [34]:
agent_executor.invoke({"input":"Tell me something about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Langsmith`


No good Wikipedia Search Result was found
Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's

{'input': 'Tell me something about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works on its own and does not require the use of LangChain. \n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using pip install -U langsmith or yarn add langchain langsmith.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by exporting necessary variables like LANGCHAIN_TRACING_V2 and LANGCHAIN_API_KEY.\n4. Log your first trace by using the provided code examples for logging traces to LangSmith.\n\nYou can learn more about LangSmith and its features on their official website.'}

In [35]:
# 2
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Machine Learning`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can analyze and make decisions or predictions based on patterns and structures in data. Machine learning is widely used in various applications such as image recognition, natural language processing, recommendation systems, and autonomous vehicles.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can analyze and make decisions or predictions based on patterns and structures in data. Machine learning is widely used in various applications such as image recognition, natural language processing, recommendation systems, and autonomous vehicles.'}

In [36]:
# 3
agent_executor.invoke({"input":"what is paper 1605.08386?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloPaper 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses graphs on lattice points where the edges come from a finite set of allocations.

> Finished chain.


{'input': 'what is paper 1605.08386?',
 'output': 'Paper 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses graphs on lattice points where the edges come from a finite set of allocations.'}